[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MaxMitre/Aplicaciones-Financieras/blob/main/Semana9/Manejo_de_portafolios_Markovitz.ipynb)

# Introducción

Trataremos de encontrar portafolios óptimos, utilizando como medidores la varianza de los retornos, así como el valor esperado de los retornos en un año (el valor que se espera ganar)

# Dependencias

In [ ]:
!pip install yfinance -U plotly

In [ ]:
import yfinance as yf

import pandas as pd
import numpy as np

import cvxopt as opt

import plotly.express as px

# Datos


Los datos con los que trabajaremos son el precio de acciones de Google (GOOG), Apple (AAPL), IBM (IBM), Microsoft (MSFT) y ExxonMobil (XOM) del último año. 

Para obtener los datos, usaremos [```yfinance```](https://github.com/ranaroussi/yfinance) y nos centraremos en el precio de cierre ajustado (Adj Close).

In [ ]:
data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "GOOG AAPL IBM MSFT XOM",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "1y",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1d",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        # group_by = 'ticker',
    ).loc[:, 'Adj Close']
data

[*********************100%***********************]  5 of 5 completed


,AAPL,GOOG,IBM,MSFT,XOM
Date,,,,,
2021-05-18,124.140762,2303.429932,131.037125,241.115997,57.506474
2021-05-19,123.981667,2308.709961,130.381516,241.712524,56.126621
2021-05-20,126.586777,2356.090088,131.009796,245.053070,55.993397
2021-05-21,124.717461,2345.100098,131.792877,243.750656,56.069527
2021-05-24,126.377975,2406.669922,131.774658,249.328201,56.726143
...,...,...,...,...,...
2022-05-11,146.500000,2279.219971,130.750000,260.549988,85.910004
2022-05-12,142.559998,2263.219971,132.899994,255.350006,86.300003
2022-05-13,147.110001,2330.310059,133.600006,261.119995,88.860001


Obtenemos los retornos logarítmicos anualizados para cada activo.

In [ ]:
annual_returns = np.log(data / data.shift()) / (1 / 252) # Para anualizar los retornos
# annual_returns

mean_returns = annual_returns.mean()
# mean_returns

cov_returns = annual_returns.cov()
cov_returns

,AAPL,GOOG,IBM,MSFT,XOM
AAPL,18.200599,12.848297,3.269063,13.338651,2.208117
GOOG,12.848297,18.770590,2.679635,14.400733,1.875316
IBM,3.269063,2.679635,12.132730,1.729321,5.288756
MSFT,13.338651,14.400733,1.729321,17.655422,0.135520
XOM,2.208117,1.875316,5.288756,0.135520,23.588489


In [ ]:
mean_returns

AAPL    0.184140
GOOG    0.013197
IBM     0.054451
MSFT    0.101296
XOM     0.471086
dtype: float64

# Markovitz

https://pyportfolioopt.readthedocs.io/en/latest/UserGuide.html

> Si $w$ es el vector de pesos de las acciones con retornos esperados $\mu$, entonces el retorno del portafolio es igual al peso de cada acción multiplicado por su retorno, es decir, $w^T \mu$. El riesgo del portafolio en términos de la matriz de covarianzas $\Sigma$ esta dado por $\sigma^2 = w^T \Sigma w$. 

> La razón de Sharpe es el retorno en exceso del portafolio por unidad de riesgo (volatilidad)

$$
SR = \frac{R_p-R_f}{\sigma}
$$

Con esto en mente, crearemos un cantidad $N$ de portafolios con pesos aleatorios y guardaremos tanto los pesos, como el retorno, la volatilidad y la razon de Sharpe para cada uno.

In [ ]:
np.random.seed(1995)

N = 10000
k = annual_returns.shape[1]

weights = np.zeros((N, k))
returns = np.zeros(N)
volatilities = np.zeros(N)
sharpe_ratios = np.zeros(N)

#weights = np.array([[1,0,0,0,0],[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1]])

#weights = np.concatenate((weights, weights2))
weights.shape

(10000, 5)

In [ ]:
for i in range(N):
    w = np.random.random(k)
    w /= np.sum(w)

    weights[i, :] = w

    returns[i] = np.dot(mean_returns, w)

    volatilities[i] = np.sqrt(np.dot(w.T, np.dot(cov_returns, w))) # w.T @ cov_returns @ w

    sharpe_ratios[i] = returns[i] / volatilities[i]

Graficamos la volatilidad contra el retorno de cada portafolio generado y coloreamos en función de la razón de Sharpe. Los portafolios (casi) óptimos, serían aquellos que tienen el mayor retorno para cierto nivel de volatilidad, o dicho de otra manera, los que tienen la menor volatilidad para algún retorno especificado.

In [ ]:
import matplotlib.pyplot as plt

px.scatter(x = volatilities, y = returns, color = sharpe_ratios,
           labels={
                     "x": "Volatilidad",
                     "y": "Retorno",
                     "color": "Razón de Sharpe"
                 }
           )
# plt.scatter(volatilities, returns, c = sharpe_ratios)

> La optimización del portafolio se puede ver como un problema de optimización convexa y una solución puede encontrarse usando programación cuadrática. Si denotamos el retorno objetivo como $\mu^*$, el problema a resolver para el portafolio sólo con posiciones largar es:

\begin{align}
    \text{min}_w && w^T \Sigma w \\
    \text{s.a.} && w^t \mu \geq \mu^*  \\
    && w^T \mathbf{1} = 1 \\
    && w_i \geq 0
\end{align}

Para resolverlo, ocuparemos la función [```covxopt.solvers.qp```](https://cvxopt.org/userguide/coneprog.html#quadratic-programming). Esta requiere que el problema de optimización se encuentre en la forma general. A saber, la forma general de un problema de programación cuadrática es la siguiente:

\begin{align}
    \text{min}_x && \frac{1}{2}x^TPx + q^Tx \\
    \text{s.a.} && Gx \preceq h \\
    && Ax = b
\end{align}

In [ ]:
mu_star = .35

In [ ]:
G = opt.matrix(-np.concatenate([mean_returns.to_numpy().reshape(1, k),np.eye(k)]), tc = 'd')
h = opt.matrix(np.concatenate([np.array([-mu_star]).reshape((1, 1)), np.zeros((k, 1))]), tc = 'd')
q = opt.matrix(0.0, (k, 1))
A = opt.matrix(1.0, (1, k))
b = opt.matrix(1.0)
P = opt.matrix(2 * cov_returns.to_numpy(), tc = 'd')

In [ ]:
results = opt.solvers.qp(P, q, G, h, A, b)
results

     pcost       dcost       gap    pres   dres
 0:  7.0439e+00  6.2331e+00  1e+01  3e+00  4e+00
 1:  7.0621e+00  6.6299e+00  2e+00  5e-01  6e-01
 2:  9.5350e+00  1.0895e+01  5e+00  4e-01  5e-01
 3:  1.1445e+01  1.1737e+01  6e-01  4e-02  5e-02
 4:  1.1997e+01  1.1992e+01  3e-02  9e-04  1e-03
 5:  1.2006e+01  1.2005e+01  1e-03  2e-05  2e-05
 6:  1.2006e+01  1.2006e+01  3e-05  2e-07  2e-07
 7:  1.2006e+01  1.2006e+01  3e-07  2e-09  2e-09
Optimal solution found.


{'dual infeasibility': 2.1119612913156378e-09,
 'dual objective': 12.005872818122002,
 'dual slack': 9.461557983829675e-09,
 'gap': 3.071611155174505e-07,
 'iterations': 7,
 'primal infeasibility': 1.6525580202130623e-09,
 'primal objective': 12.005873086771613,
 'primal slack': 1.1938474730961322e-10,
 'relative gap': 2.5584238661416843e-08,
 's': <6x1 matrix, tc='d'>,
 'status': 'optimal',
 'x': <5x1 matrix, tc='d'>,
 'y': <1x1 matrix, tc='d'>,
 'z': <6x1 matrix, tc='d'>}

Los pesos del portafolio óptimo los obtenemos de la llave ```x```

In [ ]:
w = np.asarray(results['x']).reshape((-1))
w

array([2.70130571e-01, 3.40578322e-09, 8.25175998e-03, 1.08534755e-01,
       6.13082911e-01])

In [ ]:
np.dot(mean_returns, w)

0.34999999944473076

La volatilidad de ```primal objective```

In [ ]:
np.sqrt(results['primal objective'])

3.464949218498247

Si variamos los retornos objetivos, podemos obtener la **Frontera Eficiente**, que esta constituida de los portafolios óptimos para distintos niveles del retorno.

In [ ]:
G = opt.matrix(-np.concatenate([mean_returns.to_numpy().reshape(1, k),np.eye(k)]), tc = 'd')
q = opt.matrix(0.0, (k, 1))
A = opt.matrix(1.0, (1, k))
b = opt.matrix(1.0)
P = opt.matrix(2 * cov_returns.to_numpy(), tc = 'd')

mu_stars = np.linspace(.25, .45, 20)

ws = np.zeros((len(mu_stars), k))
mus = np.zeros(len(mu_stars))
sigmas = np.zeros(len(mu_stars))

for i, mu_star in enumerate(mu_stars):
    try:
        h = opt.matrix(np.concatenate([np.array([-mu_star]).reshape((1, 1)), np.zeros((k, 1))]), tc = 'd')
        results = opt.solvers.qp(P, q, G, h, A, b)

        w = np.asarray(results['x']).reshape((-1))
        ws[i, :] = w
        mus[i] = np.dot(mean_returns, w)
        sigmas[i] = np.sqrt(results['primal objective'])
    except:
        print('domain error')

     pcost       dcost       gap    pres   dres
 0:  7.0437e+00  6.0617e+00  9e+00  3e+00  4e+00
 1:  7.0606e+00  6.4267e+00  1e+00  2e-01  3e-01
 2:  8.1126e+00  7.7123e+00  2e+00  1e-01  2e-01
 3:  8.1953e+00  8.1610e+00  6e-02  2e-03  3e-03
 4:  8.2005e+00  8.2001e+00  6e-04  2e-05  3e-05
 5:  8.2005e+00  8.2005e+00  6e-06  2e-07  3e-07
 6:  8.2005e+00  8.2005e+00  6e-08  2e-09  3e-09
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  7.0437e+00  6.0788e+00  9e+00  3e+00  4e+00
 1:  7.0607e+00  6.4438e+00  1e+00  3e-01  4e-01
 2:  8.2436e+00  7.9537e+00  2e+00  2e-01  2e-01
 3:  8.4465e+00  8.4128e+00  9e-02  4e-03  6e-03
 4:  8.4753e+00  8.4749e+00  9e-04  4e-05  6e-05
 5:  8.4756e+00  8.4756e+00  9e-06  4e-07  6e-07
 6:  8.4756e+00  8.4756e+00  9e-08  4e-09  6e-09
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  7.0437e+00  6.0961e+00  1e+01  3e+00  4e+00
 1:  7.0608e+00  6.4619e+00  1e+00  3e-01  4e-01
 2:  8.3785e+00  8.2200e

Finalmente, podemos graficar la frontera eficiente. Se puede observar que, pese a que las simulaciones nos dieron portafolios a la frontera, no eran realmente óptimos.

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_traces(
    [
        go.Scatter(
            x = volatilities, y = returns, 
            marker = dict(
                color = sharpe_ratios,
                colorbar = dict(title="Razón de Sharpe")
            ), 
            mode = 'markers', 
            showlegend = False
        ), 
        go.Scatter(
            x = sigmas, y = mus, 
            mode = 'lines + markers',  
            showlegend = False
        )
])

fig.update_layout(
    xaxis_title = 'Volatilidad',
    yaxis_title = 'Retorno'
)



Notas

1. Se están usando los retornos sin considerar la tasa libre de riesgo.

Ejercicios

1. Encontrar y graficar el portafolio óptimo de acuerdo a la razón de Sharpee 
2. Agregar los puntos de los portafolios que contienen únicamente un activo
3. Encontrar el portafolio con la menor volatilidad
4. Restar la tasa libre de riesgo a los retornos

# Mean-Variance Choice

El portafolio óptimo también se podría obtener maximizando, respecto a $w$,

$$
U(\mu, \Sigma; w) = w^T\mu - \frac{\delta}{2}w^T\Sigma w
$$

donde $\delta > 0$ es el parámetro de aversión al riesgo. La condición de primer orden para maximizarla es

$$
\mu = \delta \Sigma w
$$

lo que implica el siguiente diseño para un portafolio con riesgo:

$$
w = \left( \delta \Sigma \right)^{-1} \mu
$$

Es un sistema de ecuaciones lineales que podemos resolver con [```np.linalg.solve```](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html).

In [ ]:
delta = .2

np.linalg.solve(delta * cov_returns,  mean_returns)

array([ 0.06751794, -0.08980028, -0.03018347,  0.05305737,  0.10713654])

# Ligas interesantes

1. https://python-advanced.quantecon.org/black_litterman.html